# 🌌 GRU with Attention for Mayan Languages - NLP Project

## 📖 Project Description
This project aims to build a **translation model for Mayan languages** using a **GRU-based Seq2Seq architecture with Attention Mechanism**. It integrates **Transfer Learning** by initializing the embeddings with pre-trained embeddings from the BETO model (a BERT model for Spanish), which helps improve performance when working with low-resource languages like the ones in the Mayan language family.

## ⚙️ Architecture
The model architecture consists of:
- **Encoder:** A bidirectional GRU with two layers.
- **Attention Mechanism:** Implemented with a general attention method that scores the relevance of each encoder output for the decoder’s prediction.
- **Decoder:** A unidirectional GRU with two layers, integrated with the attention mechanism.
- **Transfer Learning:** Embeddings initialized with BETO pre-trained embeddings for the Spanish language, later fine-tuned during training.

## 📂 Dataset
The dataset used for training was obtained from the **MayanV dataset**, a curated dataset of Mayan languages released under the CC0-1.0 license.

You can access the dataset from the official repository:

- [MayanV Dataset Repository](https://github.com/transducens/mayanv/tree/master/MayanV)

### 📜 Citation

```bibtex
@inproceedings{lou-etal-2024-curated,
    title = "Curated Datasets and Neural Models for Machine Translation of Informal Registers between {M}ayan and {S}panish Vernaculars",
    author = "Lou, Andr{\'e}s  and
      P{\'e}rez-Ortiz, Juan Antonio  and
      S{\'a}nchez-Mart{\'\i}nez, Felipe  and
      S{\'a}nchez-Cartagena, V{\'\i}ctor",
    editor = "Duh, Kevin  and
      Gomez, Helena  and
      Bethard, Steven",
    booktitle = "Proceedings of the 2024 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies (Volume 1: Long Papers)",
    month = jun,
    year = "2024",
    address = "Mexico City, Mexico",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2024.naacl-long.156",
    pages = "2838--2850",
}


In [1]:
! pip install tokenizers
# Importaciones adicionales para transfer learning
!pip install transformers
from transformers import AutoModel, AutoTokenizer

In [2]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import os
import re
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# %% [markdown]
# ## 2. Dataset Exploration
#
# Let's explore the MayanV dataset structure and check what languages and data splits are available.

# %%
# Set the base path to your MayanV dataset
BASE_PATH = '/content/drive/MyDrive/MayanV'

# Function to explore the dataset structure
def explore_dataset(base_path):
    """Explore the Mayan dataset structure and count files and examples."""
    languages = os.listdir(base_path)
    languages = [lang for lang in languages if os.path.isdir(os.path.join(base_path, lang))]

    dataset_info = {}

    for lang in languages:
        lang_path = os.path.join(base_path, lang)
        splits = [split for split in os.listdir(lang_path) if os.path.isdir(os.path.join(lang_path, split))]

        lang_info = {}
        for split in splits:
            split_path = os.path.join(lang_path, split)
            es_file = os.path.join(split_path, 'data.es')
            lang_file = os.path.join(split_path, f'data.{lang}')

            # Check if both files exist
            if os.path.exists(es_file) and os.path.exists(lang_file):
                # Count lines in files
                with open(es_file, 'r', encoding='utf-8') as f:
                    es_lines = len(f.readlines())
                with open(lang_file, 'r', encoding='utf-8') as f:
                    lang_lines = len(f.readlines())

                lang_info[split] = {
                    'spanish_lines': es_lines,
                    'mayan_lines': lang_lines,
                    'matching': es_lines == lang_lines
                }

        dataset_info[lang] = lang_info

    return dataset_info

# Explore the dataset
dataset_info = explore_dataset(BASE_PATH)

Mounted at /content/drive
Using device: cuda


In [3]:
for lang, info in dataset_info.items():
    print(f"\n{lang.upper()} Language:")
    for split, split_info in info.items():
        print(f"  {split}: {split_info['spanish_lines']} Spanish lines, {split_info['mayan_lines']} Mayan lines")



KJB Language:
  train: 924 Spanish lines, 924 Mayan lines
  test: 1000 Spanish lines, 1000 Mayan lines
  dev: 1000 Spanish lines, 1000 Mayan lines

POH Language:
  test: 1000 Spanish lines, 1000 Mayan lines
  dev: 673 Spanish lines, 673 Mayan lines

KEK Language:
  test: 1000 Spanish lines, 1000 Mayan lines
  dev: 1000 Spanish lines, 1000 Mayan lines
  train: 2114 Spanish lines, 2114 Mayan lines

IXL Language:
  test: 1000 Spanish lines, 1000 Mayan lines
  dev: 1000 Spanish lines, 1000 Mayan lines
  train: 326 Spanish lines, 326 Mayan lines

ACR Language:
  dev: 343 Spanish lines, 343 Mayan lines
  test: 1000 Spanish lines, 1000 Mayan lines

MAM Language:
  dev: 1000 Spanish lines, 1000 Mayan lines
  train: 1054 Spanish lines, 1054 Mayan lines
  test: 1000 Spanish lines, 1000 Mayan lines

CAC Language:
  test: 1000 Spanish lines, 1000 Mayan lines
  dev: 1000 Spanish lines, 1000 Mayan lines
  train: 266 Spanish lines, 266 Mayan lines

ITZ Language:
  test: 1000 Spanish lines, 1000 Maya

In [4]:
def mostrar_ejemplos(idioma, split, num_ejemplos=2):
    """Muestra ejemplos de texto en español y maya para un idioma y split específicos"""
    print(f"\n=== IDIOMA: {idioma.upper()} - SPLIT: {split} ===")

    # Rutas a los archivos
    ruta_es = f"{BASE_PATH}/{idioma}/{split}/data.es"
    ruta_maya = f"{BASE_PATH}/{idioma}/{split}/data.{idioma}"

    # Intentar leer los archivos si existen
    try:
        with open(ruta_es, 'r', encoding='utf-8') as f_es:
            lineas_es = [line.strip() for line in f_es.readlines()[:num_ejemplos]]

        with open(ruta_maya, 'r', encoding='utf-8') as f_maya:
            lineas_maya = [line.strip() for line in f_maya.readlines()[:num_ejemplos]]

        # Mostrar ejemplos
        for i in range(min(num_ejemplos, len(lineas_es), len(lineas_maya))):
            print(f"  Ejemplo {i+1}:")
            print(f"    Español: {lineas_es[i]}")
            print(f"    {idioma}: {lineas_maya[i]}")

    except FileNotFoundError:
        print(f"  No se encontraron archivos para {idioma}/{split}")
    except Exception as e:
        print(f"  Error al leer archivos: {e}")

# Lista de idiomas con más datos de entrenamiento
idiomas_principales = ['tzh', 'kek', 'ttc', 'poc', 'mam', 'kjb']

# Mostrar ejemplos para cada idioma principal
for idioma in idiomas_principales:
    for split in ['train', 'dev', 'test']:
        mostrar_ejemplos(idioma, split)

# También podemos analizar estadísticas básicas de un idioma con muchos datos (tzh)
print("\n=== ESTADÍSTICAS BÁSICAS PARA TZH (IDIOMA CON MÁS DATOS) ===")

def analizar_estadisticas(idioma='tzh', split='train'):
    """Analiza estadísticas básicas de longitud de oraciones para un idioma y split"""
    # Rutas a los archivos
    ruta_es = f"{BASE_PATH}/{idioma}/{split}/data.es"
    ruta_maya = f"{BASE_PATH}/{idioma}/{split}/data.{idioma}"

    try:
        # Leer archivos
        with open(ruta_es, 'r', encoding='utf-8') as f_es:
            lineas_es = [line.strip() for line in f_es.readlines()]

        with open(ruta_maya, 'r', encoding='utf-8') as f_maya:
            lineas_maya = [line.strip() for line in f_maya.readlines()]

        # Calcular longitudes
        longitudes_es = [len(line.split()) for line in lineas_es]
        longitudes_maya = [len(line.split()) for line in lineas_maya]

        # Estadísticas básicas
        print(f"  Número total de pares de oraciones: {len(lineas_es)}")
        print(f"  Longitud promedio (palabras) en español: {sum(longitudes_es)/len(longitudes_es):.2f}")
        print(f"  Longitud promedio (palabras) en {idioma}: {sum(longitudes_maya)/len(longitudes_maya):.2f}")
        print(f"  Longitud máxima en español: {max(longitudes_es)}")
        print(f"  Longitud máxima en {idioma}: {max(longitudes_maya)}")

        # Ejemplos de oraciones cortas y largas
        idx_corta_es = longitudes_es.index(min(longitudes_es))
        idx_larga_es = longitudes_es.index(max(longitudes_es))

        print("\n  Ejemplo de oración más corta:")
        print(f"    Español: {lineas_es[idx_corta_es]}")
        print(f"    {idioma}: {lineas_maya[idx_corta_es]}")

        print("\n  Ejemplo de oración más larga:")
        print(f"    Español: {lineas_es[idx_larga_es]}")
        print(f"    {idioma}: {lineas_maya[idx_larga_es]}")

    except Exception as e:
        print(f"  Error al analizar estadísticas: {e}")

# Analizar estadísticas para tzh
analizar_estadisticas('tzh', 'train')


=== IDIOMA: TZH - SPLIT: train ===
  Ejemplo 1:
    Español: #tzh# Tengo bocio
    tzh: Ay jtsuhnukʼ
  Ejemplo 2:
    Español: #tzh# Subió el valor del dinero
    tzh: Moj te yajtalul te takʼine

=== IDIOMA: TZH - SPLIT: dev ===
  Ejemplo 1:
    Español: #tzh# No cepilló bien la tabla
    tzh: Maba la xchʼul ta lek me chʼuhteʼe
  Ejemplo 2:
    Español: #tzh# El señor acarrea cosas
    tzh: Ya xʼeaj te jtatike

=== IDIOMA: TZH - SPLIT: test ===
  Ejemplo 1:
    Español: #tzh# El niño tiene la panza resaltada
    tzh: Terel xchʼujt alal
  Ejemplo 2:
    Español: #tzh# Está claro lo que dice
    tzh: Jahtalal chikan te binti ya kyale

=== IDIOMA: KEK - SPLIT: train ===
  Ejemplo 1:
    Español: #kek# La mujer contrajo mal de ojos
    kek: Xbʼeʼirk seʼ ru li ixq
  Ejemplo 2:
    Español: #kek# Se paró delante de mí
    kek: Xxaqabʼ ribʼ chi wu

=== IDIOMA: KEK - SPLIT: dev ===
  Ejemplo 1:
    Español: #kek# Fue aclarándose mi vista cuando me curé
    kek: Naʼajajnak saʼ wu naq xinkʼira


In [5]:
import os
import re
import random
import time
import math
import unicodedata
import string
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [6]:
# Definir configuración del modelo paso 1
# Definir configuración del modelo
class Config:
    # Parámetros del modelo
    EMBEDDING_DIM = 256
    HIDDEN_DIM = 256
    N_LAYERS = 2
    DROPOUT = 0.4  # Aumentado para reducir sobreajuste

    # Parámetros de entrenamiento
    BATCH_SIZE = 64
    LEARNING_RATE = 0.0005
    WEIGHT_DECAY = 0.1  # Regularización más fuerte
    NUM_EPOCHS = 40
    EARLY_STOP_PATIENCE = 10  # Más paciencia para considerar mejoras en BLEU
    CLIP = 1.0
    TEACHER_FORCING_RATIO = 0.7  # Más teacher forcing

    # Parámetros para BPE
    VOCAB_SIZE_ES = 10000   # Mantener español con vocabulario grande
    VOCAB_SIZE_MAYA = 8000  # Reducir para maya (mejor cobertura)
    MIN_FREQ = 2

    # Parámetros generales
    SEED = 42
    MAX_LENGTH = 50

    # Idioma a trabajar inicialmente
    LANGUAGE = 'tzh'     # Tzotzil

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Configurar semillas para reproducibilidad
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(Config.SEED)

In [8]:
# Paso 2-3: Preprocesamiento y Tokenizador BPE con Data Augmentation
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, processors
import json

from tokenizers import Tokenizer, models, pre_tokenizers, trainers, processors
import json
# Importar para transfer learning
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn

def normalize_string(s):
    """Elimina etiquetas especiales del texto y normaliza"""
    # Eliminar etiquetas como #tzh#
    s = re.sub(r'#\w+#\s*', '', s)
    return s.strip()

def augment_data(text):
    """Aplica técnicas sencillas de data augmentation al texto en español"""
    words = text.split()

    # Solo aplicar a oraciones suficientemente largas y con cierta probabilidad
    if len(words) <= 3 or random.random() > 0.3:
        return text

    # Técnica 1: Intercambio de palabras adyacentes
    if random.random() < 0.5 and len(words) > 3:
        i = random.randint(0, len(words) - 2)
        words[i], words[i+1] = words[i+1], words[i]

    # Técnica 2: Eliminación aleatoria de una palabra (simular dropout léxico)
    elif len(words) > 4 and random.random() < 0.4:
        # Evitar eliminar la primera o última palabra
        i = random.randint(1, len(words) - 2)
        words.pop(i)

    return ' '.join(words)

def load_data(base_path, language, split):
    """Carga datos de un idioma y split específicos con augmentación para entrenamiento"""
    lang_path = os.path.join(base_path, language, split)
    es_file = os.path.join(lang_path, 'data.es')
    lang_file = os.path.join(lang_path, f'data.{language}')

    # Verificar si los archivos existen
    if not os.path.exists(es_file) or not os.path.exists(lang_file):
        return [], []

    # Leer archivos
    with open(es_file, 'r', encoding='utf-8') as f:
        es_lines = [normalize_string(line.strip()) for line in f.readlines()]

    with open(lang_file, 'r', encoding='utf-8') as f:
        maya_lines = [line.strip() for line in f.readlines()]

    # Asegurar mismo número de líneas
    if len(es_lines) != len(maya_lines):
        min_lines = min(len(es_lines), len(maya_lines))
        es_lines = es_lines[:min_lines]
        maya_lines = maya_lines[:min_lines]

    # Aplicar aumentación solo al conjunto de entrenamiento
    if split == 'train':
        # Generar versiones aumentadas
        augmented_es = []
        augmented_maya = []

        for src, tgt in zip(es_lines, maya_lines):
            # Añadir par original
            augmented_es.append(src)
            augmented_maya.append(tgt)

            # Añadir versión aumentada (30% de probabilidad)
            if random.random() < 0.3:
                aug_src = augment_data(src)
                if aug_src != src:  # Solo si realmente cambió
                    augmented_es.append(aug_src)
                    augmented_maya.append(tgt)

        print(f"Datos originales: {len(es_lines)}, Datos después de aumentación: {len(augmented_es)}")
        return augmented_es, augmented_maya

    return es_lines, maya_lines

def train_bpe_tokenizer(texts, vocab_size=8000, min_frequency=2, save_path=None):
    """Entrena un tokenizador BPE en los textos dados"""
    # Inicializar tokenizador
    tokenizer = Tokenizer(models.BPE())

    # Configurar pre-tokenizador (por palabras)
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

    # Configurar entrenador - AQUÍ USAMOS TOKENS SIN ESPACIOS INTERNOS
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        min_frequency=min_frequency,
        special_tokens=["<PAD>", "[SOS]", "[EOS]", "<UNK>"]
    )

    # Entrenar tokenizador
    tokenizer.train_from_iterator(texts, trainer)

    # Añadir post-procesador para manejar tokens especiales - CONSISTENTE CON LOS NOMBRES ARRIBA
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[SOS] $A [EOS]",
        special_tokens=[
            ("[SOS]", tokenizer.token_to_id("[SOS]")),
            ("[EOS]", tokenizer.token_to_id("[EOS]"))
        ],
    )

    # Guardar tokenizador si se proporciona una ruta
    if save_path:
        tokenizer.save(save_path)

    return tokenizer

class BPEVocabulary:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.index2word = {}

        # Preparar mapeos inversos
        vocab = json.loads(tokenizer.to_str())["model"]["vocab"]
        for token, id in vocab.items():
            self.index2word[id] = token

        # Obtener IDs para tokens especiales
        self.pad_id = tokenizer.token_to_id("<PAD>")
        self.sos_id = tokenizer.token_to_id("[SOS]")
        self.eos_id = tokenizer.token_to_id("[EOS]")
        self.unk_id = tokenizer.token_to_id("<UNK>")

        # Tamaño del vocabulario
        self.n_words = len(vocab)

    def encode(self, text):
        """Codifica texto a lista de IDs de tokens"""
        encoding = self.tokenizer.encode(text)
        return encoding.ids

    def decode(self, ids):
        """Decodifica lista de IDs a texto"""
        return self.tokenizer.decode(ids)

def load_pretrained_spanish_embeddings(bpe_vocab, embedding_dim=Config.EMBEDDING_DIM):
    """
    Carga embeddings preentrenados de BETO y los mapea al vocabulario BPE del español
    """
    print("Cargando modelo preentrenado BETO...")
    # Cargar modelo y tokenizador preentrenados
    pretrained_model_name = "dccuchile/bert-base-spanish-wwm-cased"
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
    model = AutoModel.from_pretrained(pretrained_model_name)

    # Obtener matriz de embeddings original
    pretrained_embeddings = model.embeddings.word_embeddings.weight.data
    pretrained_vocab_size, pretrained_dim = pretrained_embeddings.shape

    print(f"Modelo preentrenado cargado: {pretrained_vocab_size} tokens, {pretrained_dim} dimensiones")

    # Crear matriz de embeddings para nuestro vocabulario BPE
    target_vocab_size = bpe_vocab.n_words
    mapped_embeddings = torch.FloatTensor(target_vocab_size, embedding_dim).normal_(0, 0.1)

    # Proyector para reducir dimensionalidad si es necesario
    if pretrained_dim != embedding_dim:
        print(f"Proyectando embeddings de {pretrained_dim} a {embedding_dim} dimensiones...")
        projector = nn.Linear(pretrained_dim, embedding_dim)
        nn.init.xavier_uniform_(projector.weight.data)
        with torch.no_grad():
            pretrained_embeddings = projector(pretrained_embeddings)

    # Mapear tokens de nuestro vocabulario a embeddings preentrenados
    print(f"Mapeando vocabulario BPE ({target_vocab_size} tokens) a embeddings preentrenados...")

    # Tokens especiales primero
    special_tokens = ["<PAD>", "< SOS >", "<EOS>", "<UNK>"]
    for token in special_tokens:
        if token in tokenizer.vocab:
            idx_pretrained = tokenizer.vocab[token]
            idx_our = bpe_vocab.tokenizer.token_to_id(token)
            if idx_our is not None:
                mapped_embeddings[idx_our] = pretrained_embeddings[idx_pretrained]

    # Resto del vocabulario
    words_mapped = 0
    for token, idx_our in json.loads(bpe_vocab.tokenizer.to_str())["model"]["vocab"].items():
        # Ignorar tokens especiales ya mapeados
        if token in special_tokens:
            continue

        # Intentar encontrar el token en el vocabulario preentrenado
        if token in tokenizer.vocab:
            # Mapeo directo
            idx_pretrained = tokenizer.vocab[token]
            mapped_embeddings[idx_our] = pretrained_embeddings[idx_pretrained]
            words_mapped += 1
        elif token.strip() in tokenizer.vocab:
            # Intentar sin espacios
            idx_pretrained = tokenizer.vocab[token.strip()]
            mapped_embeddings[idx_our] = pretrained_embeddings[idx_pretrained]
            words_mapped += 1
        elif len(token) > 2 and token[0] == "▁":
            # Tokens tipo SentencePiece que empiezan con ▁
            clean_token = token[1:]
            if clean_token in tokenizer.vocab:
                idx_pretrained = tokenizer.vocab[clean_token]
                mapped_embeddings[idx_our] = pretrained_embeddings[idx_pretrained]
                words_mapped += 1

    # Reportar estadísticas de mapeo
    print(f"Tokens mapeados: {words_mapped}/{target_vocab_size} ({words_mapped/target_vocab_size*100:.1f}%)")

    return mapped_embeddings

In [9]:
# Paso 4: Dataset y DataLoader con BPE
class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data, src_vocab, tgt_vocab, max_length=Config.MAX_LENGTH):
        self.src_data = src_data
        self.tgt_data = tgt_data
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_text = self.src_data[idx]
        tgt_text = self.tgt_data[idx]

        # Obtener tokens codificados con BPE
        src_ids = self.src_vocab.encode(src_text)
        tgt_ids = self.tgt_vocab.encode(tgt_text)

        # Truncar si es demasiado largo
        if len(src_ids) > self.max_length:
            src_ids = src_ids[:self.max_length]

        if len(tgt_ids) > self.max_length:
            tgt_ids = tgt_ids[:self.max_length]

        return torch.LongTensor(src_ids), torch.LongTensor(tgt_ids)

def collate_fn(batch):
    """Función para agrupar elementos en batch con padding"""
    src_batch, tgt_batch = zip(*batch)

    # Padding de secuencias
    src_batch_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_batch_padded, tgt_batch_padded

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, n_layers,
                         dropout=dropout, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.n_layers = n_layers

    def forward(self, src):
        # src: [batch_size, src_len]
        embedded = self.dropout(self.embedding(src))
        # embedded: [batch_size, src_len, embedding_dim]

        outputs, hidden = self.rnn(embedded)
        # outputs: [batch_size, src_len, hidden_dim*2] (bidireccional)
        # hidden: [n_layers*2, batch_size, hidden_dim] (bidireccional)

        # Reorganizar el estado oculto para que tenga la forma correcta
        # Separar direcciones y procesar cada capa
        hidden_forward = hidden[0:hidden.size(0):2]
        hidden_backward = hidden[1:hidden.size(0):2]

        # Concatenar y transformar para cada capa
        hidden_transformed = []
        for i in range(self.n_layers):
            combined = torch.cat((hidden_forward[i], hidden_backward[i]), dim=1)
            transformed = torch.tanh(self.fc(combined))
            hidden_transformed.append(transformed.unsqueeze(0))

        # Concatenar a lo largo de la dimensión de capas
        final_hidden = torch.cat(hidden_transformed, dim=0)
        # final_hidden: [n_layers, batch_size, hidden_dim]

        return outputs, final_hidden

In [11]:
# Paso 6: Modelo - Mecanismo de atención
class Attention(nn.Module):
    def __init__(self, hidden_dim, method='general'):
        super().__init__()
        self.method = method
        self.hidden_dim = hidden_dim

        if self.method == 'general':
            self.attn = nn.Linear(hidden_dim*2, hidden_dim)
            self.v = nn.Linear(hidden_dim, 1, bias=False)
        elif self.method == 'concat':
            self.attn = nn.Linear(hidden_dim*3, hidden_dim)
            self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [1, batch_size, hidden_dim]
        # encoder_outputs: [batch_size, src_len, hidden_dim*2]

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        # Repetir hidden para que coincida con las dimensiones de encoder_outputs
        hidden = hidden.transpose(0, 1)  # [batch_size, 1, hidden_dim]
        hidden = hidden.repeat(1, src_len, 1)  # [batch_size, src_len, hidden_dim]

        if self.method == 'general':
            # Calcular energía
            energy = torch.tanh(self.attn(encoder_outputs))  # [batch_size, src_len, hidden_dim]
            energy = self.v(energy).squeeze(2)  # [batch_size, src_len]

        elif self.method == 'concat':
            # Concatenar hidden y encoder_outputs
            energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
            energy = self.v(energy).squeeze(2)  # [batch_size, src_len]

        # Convertir a pesos de atención
        return F.softmax(energy, dim=1)  # [batch_size, src_len]

In [12]:
# Paso 7: Modelo - Decoder con atención
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim + hidden_dim*2, hidden_dim, n_layers,
                         dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim*3 + embedding_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size]
        # hidden: [1, batch_size, hidden_dim]
        # encoder_outputs: [batch_size, src_len, hidden_dim*2]

        input = input.unsqueeze(1)  # [batch_size, 1]

        embedded = self.dropout(self.embedding(input))  # [batch_size, 1, embedding_dim]

        # Calcular pesos de atención
        attn_weights = self.attention(hidden, encoder_outputs)  # [batch_size, src_len]
        attn_weights = attn_weights.unsqueeze(1)  # [batch_size, 1, src_len]

        # Aplicar atención a las salidas del encoder
        context = torch.bmm(attn_weights, encoder_outputs)  # [batch_size, 1, hidden_dim*2]

        # Concatenar embedded y context
        rnn_input = torch.cat((embedded, context), dim=2)  # [batch_size, 1, embedding_dim + hidden_dim*2]

        # Pasar por el RNN
        output, hidden = self.rnn(rnn_input, hidden)  # [batch_size, 1, hidden_dim], [1, batch_size, hidden_dim]

        # Concatenar output, embedded y context para predicción
        embedded = embedded.squeeze(1)  # [batch_size, embedding_dim]
        output = output.squeeze(1)  # [batch_size, hidden_dim]
        context = context.squeeze(1)  # [batch_size, hidden_dim*2]

        # Predicción final
        prediction = self.fc_out(torch.cat((output, context, embedded), dim=1))
        # prediction: [batch_size, output_dim]

        return prediction, hidden

In [13]:
# Paso 8: Modelo - Seq2Seq completo
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=Config.TEACHER_FORCING_RATIO):
        # src: [batch_size, src_len]
        # tgt: [batch_size, tgt_len]

        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.output_dim

        # Tensor para almacenar salidas del decoder
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Codificar secuencia de entrada
        encoder_outputs, hidden = self.encoder(src)

        # Primera entrada al decoder es el token <SOS>
        input = tgt[:, 0]

        for t in range(1, tgt_len):
            # Decodificar
            output, hidden = self.decoder(input, hidden, encoder_outputs)

            # Almacenar salida
            outputs[:, t, :] = output

            # Determinar si usamos teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Obtener el token con mayor probabilidad
            top1 = output.argmax(1)

            # Usar teacher forcing o token predicho
            input = tgt[:, t] if teacher_force else top1

        return outputs

In [14]:
# Paso 9: Funciones de entrenamiento y evaluación

def calculate_accuracy(output, target, pad_idx=0):
    """Calcula la precisión de las predicciones ignorando padding"""
    pred = output.argmax(1)
    non_pad = target.ne(pad_idx)
    correct = pred.eq(target).masked_select(non_pad).sum().item()
    total = non_pad.sum().item()
    return correct / total if total > 0 else 0

def calculate_perplexity(loss):
    """Calcula la perplejidad a partir de la pérdida"""
    return math.exp(loss)

def train(model, dataloader, optimizer, criterion, clip, device):
    """Función de entrenamiento por época con técnicas anti-sobreajuste"""
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for src, tgt in tqdm(dataloader, desc="Entrenando"):
        src = src.to(device)
        tgt = tgt.to(device)

        optimizer.zero_grad()

        # Teacher forcing con ratio actualizado en Config
        output = model(src, tgt, Config.TEACHER_FORCING_RATIO)

        # Reformar output y target para cálculo de pérdida
        output_dim = output.shape[-1]
        # Ignorar posición del token < SOS >
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)

        # Aplicar la función de pérdida (ahora con label smoothing)
        loss = criterion(output, tgt)
        loss.backward()

        # Recortar gradientes
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        # Calcular métricas
        acc = calculate_accuracy(output, tgt)

        epoch_loss += loss.item()
        epoch_acc += acc

    avg_loss = epoch_loss / len(dataloader)
    avg_acc = epoch_acc / len(dataloader)

    return {
        'loss': avg_loss,
        'accuracy': avg_acc,
        'perplexity': calculate_perplexity(avg_loss)
    }

def evaluate(model, dataloader, criterion, device):
    """Función de evaluación mejorada con mejor cálculo de BLEU"""
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for src, tgt in tqdm(dataloader, desc="Evaluando"):
            src = src.to(device)
            tgt = tgt.to(device)

            output = model(src, tgt, 0)  # Desactivar teacher forcing

            # Reformar output y target para cálculo de pérdida
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)

            # Calcular métricas
            acc = calculate_accuracy(output, tgt)

            epoch_loss += loss.item()
            epoch_acc += acc

    avg_loss = epoch_loss / len(dataloader)
    avg_acc = epoch_acc / len(dataloader)

    # Calcular BLEU score en un conjunto más amplio y balanceado de ejemplos
    bleu_score = 0
    try:
        from nltk.translate.bleu_score import corpus_bleu
        import random

        # Aumentar tamaño de muestras para BLEU si conjunto no es muy grande
        sample_size = min(200, len(dataloader.dataset))
        indices = random.sample(range(len(dataloader.dataset)), sample_size)
        sample_translations = []
        sample_references = []

        for idx in indices:
            src_text = dataloader.dataset.src_data[idx]
            tgt_text = dataloader.dataset.tgt_data[idx]

            translation = translate_sentence(src_text, dataloader.dataset.src_vocab,
                                            dataloader.dataset.tgt_vocab, model, device)

            # Normalizar texto para comparación BLEU
            translation_tokens = translation.lower().split()
            reference_tokens = tgt_text.lower().split()

            sample_translations.append(translation_tokens)
            sample_references.append([reference_tokens])

        # Usar corpus_bleu para mayor estabilidad
        bleu_score = corpus_bleu(sample_references, sample_translations)

        # Reportar muestras evaluadas
        print(f"BLEU calculado sobre {sample_size} muestras aleatorias")
    except Exception as e:
        print(f"No se pudo calcular BLEU: {e}")
        bleu_score = 0

    return {
        'loss': avg_loss,
        'accuracy': avg_acc,
        'perplexity': calculate_perplexity(avg_loss),
        'bleu': bleu_score
    }

def epoch_time(start_time, end_time):
    """Calcular tiempo transcurrido entre épocas"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Función auxiliar para monitorear el ratio de sobreajuste
def calculate_overfitting_ratio(train_loss, valid_loss):
    """Calcula el ratio de sobreajuste para monitoreo"""
    if valid_loss == 0:
        return float('inf')
    return valid_loss / train_loss

In [15]:
# Paso 10: Traducción con BPE
def translate_sentence(sentence, src_vocab, tgt_vocab, model, device, max_length=Config.MAX_LENGTH):
    """Traducir una oración del español a la lengua maya usando BPE"""
    model.eval()

    # Normalizar y tokenizar con BPE
    sentence = normalize_string(sentence)
    src_ids = src_vocab.encode(sentence)

    # Truncar si es demasiado largo
    if len(src_ids) > max_length:
        src_ids = src_ids[:max_length]

    # Convertir a tensor
    src_tensor = torch.LongTensor(src_ids).unsqueeze(0).to(device)

    # Obtener salidas del encoder
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

    # Iniciar con token SOS
    input = torch.tensor([tgt_vocab.sos_id]).to(device)

    generated_ids = []

    for _ in range(max_length):
        with torch.no_grad():
            output, hidden = model.decoder(input, hidden, encoder_outputs)

        # Obtener token predicho
        pred_token = output.argmax(1).item()

        # Detener si es EOS
        if pred_token == tgt_vocab.eos_id:
            break

        # Añadir a lista de IDs generados
        generated_ids.append(pred_token)

        # Actualizar entrada
        input = torch.tensor([pred_token]).to(device)

    # Decodificar tokens a texto
    return tgt_vocab.decode(generated_ids)

In [16]:
# Paso 11: Pipeline completo con BPE y Early Stopping
def train_mayan_translator(base_path, language=Config.LANGUAGE):
    """Pipeline completo para entrenar un traductor español-maya con BPE, transfer learning y técnicas anti-sobreajuste"""
    print(f"Entrenando traductor para Español <-> {language}")

    # Cargar datos con data augmentation
    train_es, train_maya = load_data(base_path, language, 'train')
    dev_es, dev_maya = load_data(base_path, language, 'dev')
    test_es, test_maya = load_data(base_path, language, 'test')

    print(f"Datos cargados: {len(train_es)} ejemplos de entrenamiento, "
          f"{len(dev_es)} de desarrollo, {len(test_es)} de prueba")

    # Configurar directorios para tokenizadores
    save_dir = f'tokenizers_{language}'
    os.makedirs(save_dir, exist_ok=True)

    # Entrenar tokenizadores BPE (con tamaños diferenciados)
    print("Entrenando tokenizador BPE para español...")
    es_tokenizer = train_bpe_tokenizer(
        train_es,
        vocab_size=Config.VOCAB_SIZE_ES,  # Mantener tamaño original para español
        min_frequency=Config.MIN_FREQ,
        save_path=os.path.join(save_dir, 'spanish_bpe.json')
    )

    print("Entrenando tokenizador BPE para lengua maya...")
    maya_tokenizer = train_bpe_tokenizer(
        train_maya,
        vocab_size=Config.VOCAB_SIZE_MAYA,  # Usar tamaño reducido para maya
        min_frequency=Config.MIN_FREQ,
        save_path=os.path.join(save_dir, f'{language}_bpe.json')
    )

    # Crear vocabularios a partir de tokenizadores
    es_vocab = BPEVocabulary(es_tokenizer)
    maya_vocab = BPEVocabulary(maya_tokenizer)

    print(f"Tamaño de vocabulario español: {es_vocab.n_words}")
    print(f"Tamaño de vocabulario {language}: {maya_vocab.n_words}")

    # Crear datasets
    train_dataset = TranslationDataset(train_es, train_maya, es_vocab, maya_vocab)
    dev_dataset = TranslationDataset(dev_es, dev_maya, es_vocab, maya_vocab)
    test_dataset = TranslationDataset(test_es, test_maya, es_vocab, maya_vocab)

    # Crear dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn
    )

    dev_loader = DataLoader(
        dev_dataset,
        batch_size=Config.BATCH_SIZE,
        collate_fn=collate_fn
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=Config.BATCH_SIZE,
        collate_fn=collate_fn
    )

    # Crear componentes del modelo
    encoder = Encoder(
        input_dim=es_vocab.n_words,
        embedding_dim=Config.EMBEDDING_DIM,
        hidden_dim=Config.HIDDEN_DIM,
        n_layers=Config.N_LAYERS,
        dropout=Config.DROPOUT
    )

    # NUEVO: Inicializar con embeddings preentrenados
    try:
        print("Inicializando embeddings del encoder con modelo preentrenado BETO...")
        pretrained_embeddings = load_pretrained_spanish_embeddings(es_vocab)
        # Copiar embeddings preentrenados a la capa de embedding del encoder
        with torch.no_grad():
            encoder.embedding.weight.copy_(pretrained_embeddings)
            # Congelar los embeddings inicialmente
            encoder.embedding.weight.requires_grad = False
        print("Embeddings del encoder inicializados con éxito y congelados para entrenamiento inicial")
    except Exception as e:
        print(f"No se pudieron cargar los embeddings preentrenados: {e}")
        print("Continuando con embeddings aleatorios")

    attention = Attention(
        hidden_dim=Config.HIDDEN_DIM
    )

    decoder = Decoder(
        output_dim=maya_vocab.n_words,
        embedding_dim=Config.EMBEDDING_DIM,
        hidden_dim=Config.HIDDEN_DIM,
        n_layers=Config.N_LAYERS,
        dropout=Config.DROPOUT,
        attention=attention
    )

    # Crear modelo Seq2Seq
    model = Seq2Seq(encoder, decoder, Config.DEVICE).to(Config.DEVICE)

    # Inicializar parámetros (excepto los embeddings que ya están preentrenados)
    def initialize_weights(m):
        if hasattr(m, 'weight') and m.weight.dim() > 1:
            # No reinicializar la capa de embedding del encoder
            if isinstance(m, nn.Embedding) and m == encoder.embedding:
                return
            nn.init.xavier_uniform_(m.weight.data)

    model.apply(initialize_weights)

    # Mostrar número de parámetros
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"El modelo tiene {num_params:,} parámetros entrenables")

    # Definir optimizador con tasa de aprendizaje diferencial para embeddings
    # Separar parámetros para usar diferentes learning rates
    encoder_embedding_params = list(model.encoder.embedding.parameters())
    rest_params = [p for n, p in model.named_parameters() if not n.startswith('encoder.embedding')]

    optimizer = optim.AdamW([
        {'params': rest_params},
        {'params': encoder_embedding_params, 'lr': Config.LEARNING_RATE * 0.1}  # Learning rate menor para embeddings
    ], lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)

    # Scheduler con tasa de aprendizaje adaptativa
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=2,
        verbose=True,
        min_lr=1e-6
    )

    # Ignorar índice de padding (0) en cálculo de pérdida + LABEL SMOOTHING
    criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)

    # Entrenamiento con monitoreo dual (loss y BLEU)
    best_valid_loss = float('inf')
    best_bleu = 0.0
    early_stop_counter = 0

    # Para seguimiento de métricas
    train_metrics_history = {'loss': [], 'accuracy': [], 'perplexity': []}
    valid_metrics_history = {'loss': [], 'accuracy': [], 'perplexity': [], 'bleu': []}

    # Variables para descongelamiento gradual
    embeddings_unfrozen = False

    for epoch in range(Config.NUM_EPOCHS):
        start_time = time.time()

        # Descongelar embeddings después de 5 épocas
        if epoch == 5 and not embeddings_unfrozen:
            print("Descongelando embeddings preentrenados para ajuste fino...")
            for param in model.encoder.embedding.parameters():
                param.requires_grad = True
            embeddings_unfrozen = True

            # Actualizar contador de parámetros entrenables
            num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Embeddings descongelados. El modelo ahora tiene {num_params:,} parámetros entrenables")

        # Entrenar y evaluar modelo
        train_metrics = train(model, train_loader, optimizer, criterion, Config.CLIP, Config.DEVICE)
        valid_metrics = evaluate(model, dev_loader, criterion, Config.DEVICE)

        # Actualizar learning rate scheduler
        scheduler.step(valid_metrics['loss'])

        # Guardar métricas para análisis posterior
        for metric in train_metrics:
            train_metrics_history[metric].append(train_metrics[metric])

        for metric in valid_metrics:
            valid_metrics_history[metric].append(valid_metrics[metric])

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Calcular ratio de sobreajuste
        overfitting_ratio = train_metrics['loss'] / valid_metrics['loss'] if valid_metrics['loss'] > 0 else 0

        print(f"Época {epoch+1}/{Config.NUM_EPOCHS} | Tiempo: {epoch_mins}m {epoch_secs}s")
        print(f"Pérdida de entrenamiento: {train_metrics['loss']:.3f} | Pérdida de validación: {valid_metrics['loss']:.3f}")
        print(f"Precisión entrenamiento: {train_metrics['accuracy']:.3f} | Precisión validación: {valid_metrics['accuracy']:.3f}")
        print(f"Perplejidad entrenamiento: {train_metrics['perplexity']:.3f} | Perplejidad validación: {valid_metrics['perplexity']:.3f}")
        print(f"BLEU validación: {valid_metrics['bleu']:.4f} | Ratio de sobreajuste: {overfitting_ratio:.2f}")

        # Detectar sobreajuste (umbral más estricto)
        if epoch > 2 and overfitting_ratio < 0.5:
            print("Alerta: Sobreajuste severo detectado (ratio entrenamiento/validación < 0.5)")
        elif epoch > 2 and overfitting_ratio < 0.7:
            print("Alerta: Posible sobreajuste detectado (ratio entrenamiento/validación < 0.7)")

        # Variable para seguir si hubo alguna mejora
        improved = False

        # Guardar modelo basado en mejor pérdida de validación
        if valid_metrics['loss'] < best_valid_loss:
            best_valid_loss = valid_metrics['loss']
            torch.save(model.state_dict(), f'model_{language}_best_loss.pt')
            print(f"Modelo guardado con mejor pérdida de validación: {valid_metrics['loss']:.3f}")
            improved = True

        # Guardar modelo basado en mejor BLEU
        if valid_metrics['bleu'] > best_bleu:
            best_bleu = valid_metrics['bleu']
            torch.save(model.state_dict(), f'model_{language}_best_bleu.pt')
            print(f"Modelo guardado con mejor BLEU: {valid_metrics['bleu']:.4f}")
            improved = True

        # Actualizar contador de early stopping basado en cualquier mejora
        if improved:
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        # Verificar early stopping
        if early_stop_counter >= Config.EARLY_STOP_PATIENCE:
            print(f"Early stopping después de {Config.EARLY_STOP_PATIENCE} épocas sin mejora en ninguna métrica.")
            break

        # Traducir algunos ejemplos
        if (epoch + 1) % 5 == 0 or epoch == Config.NUM_EPOCHS - 1:
            print("\nEjemplos de traducción:")
            for i in range(min(3, len(test_es))):
                src = test_es[i]
                tgt = test_maya[i]
                translation = translate_sentence(src, es_vocab, maya_vocab, model, Config.DEVICE)

                print(f"Fuente: {src}")
                print(f"Destino: {tgt}")
                print(f"Predicción: {translation}")
                print()

    # Evaluar ambos modelos en conjunto de prueba
    print("\nEvaluando modelo con mejor pérdida:")
    model.load_state_dict(torch.load(f'model_{language}_best_loss.pt'))
    test_metrics_loss = evaluate(model, test_loader, criterion, Config.DEVICE)

    print("\nEvaluando modelo con mejor BLEU:")
    model.load_state_dict(torch.load(f'model_{language}_best_bleu.pt'))
    test_metrics_bleu = evaluate(model, test_loader, criterion, Config.DEVICE)

    # Mostrar resultados comparativos
    print(f"\nResultados en conjunto de prueba:")
    print(f"  Modelo con mejor loss: Loss={test_metrics_loss['loss']:.3f}, BLEU={test_metrics_loss['bleu']:.4f}")
    print(f"  Modelo con mejor BLEU: Loss={test_metrics_bleu['loss']:.3f}, BLEU={test_metrics_bleu['bleu']:.4f}")

    # Seleccionar el mejor modelo final (priorizando BLEU en prueba)
    final_model_path = f'model_{language}_best_bleu.pt'
    if test_metrics_bleu['bleu'] >= test_metrics_loss['bleu']:
        print(f"\nModelo final: modelo con mejor BLEU")
        test_metrics = test_metrics_bleu
    else:
        print(f"\nModelo final: modelo con mejor loss")
        final_model_path = f'model_{language}_best_loss.pt'
        test_metrics = test_metrics_loss

    # Copiar el mejor modelo a la ubicación estándar para compatibilidad
    import shutil
    shutil.copy(final_model_path, f'model_{language}.pt')

    # Guardar vocabularios y métricas
    save_path = f'translator_{language}'
    os.makedirs(save_path, exist_ok=True)

    # Guardar métricas históricas para análisis posterior
    try:
        import json
        with open(os.path.join(save_path, "metrics_history.json"), 'w', encoding='utf-8') as f:
            json.dump({
                'train': train_metrics_history,
                'valid': valid_metrics_history,
                'test_best_loss': test_metrics_loss,
                'test_best_bleu': test_metrics_bleu,
                'final_test': test_metrics,
                'config': {
                    'vocab_size_es': Config.VOCAB_SIZE_ES,
                    'vocab_size_maya': Config.VOCAB_SIZE_MAYA,
                    'pretrained_embeddings': True,
                    'embeddings_unfrozen': embeddings_unfrozen
                }
            }, f, indent=2)
    except Exception as e:
        print(f"No se pudieron guardar métricas históricas en formato JSON: {e}")

    # Imprimir mejoras respecto al modelo base
    print("\nMejoras con técnicas anti-sobreajuste y transfer learning:")

    # Comparar métricas del modelo seleccionado
    print(f"  BLEU final: {test_metrics['bleu']:.4f}")
    print(f"  Accuracy final: {test_metrics['accuracy']:.3f}")
    print(f"  Perplejidad final: {test_metrics['perplexity']:.3f}")

    # Reportar ejemplos de traducción del modelo final
    print("\nEjemplos de traducción del modelo final:")
    model.load_state_dict(torch.load(f'model_{language}.pt'))
    for i in range(min(5, len(test_es))):
        src = test_es[i]
        tgt = test_maya[i]
        translation = translate_sentence(src, es_vocab, maya_vocab, model, Config.DEVICE)

        print(f"Fuente: {src}")
        print(f"Destino: {tgt}")
        print(f"Predicción: {translation}")
        print()

    return {
        'model': model,
        'es_vocab': es_vocab,
        'maya_vocab': maya_vocab,
        'test_metrics': test_metrics,
        'train_history': train_metrics_history,
        'valid_history': valid_metrics_history
    }


In [17]:
# Ejecutar función principal
if __name__ == "__main__":
    print(f"Dispositivo: {Config.DEVICE}")
    results = train_mayan_translator(BASE_PATH, Config.LANGUAGE)
    print("Entrenamiento completado!")

Dispositivo: cuda
Entrenando traductor para Español <-> tzh
Datos originales: 17846, Datos después de aumentación: 18845
Datos cargados: 18845 ejemplos de entrenamiento, 1000 de desarrollo, 1000 de prueba
Entrenando tokenizador BPE para español...
Entrenando tokenizador BPE para lengua maya...
Tamaño de vocabulario español: 9724
Tamaño de vocabulario tzh: 8000
Inicializando embeddings del encoder con modelo preentrenado BETO...
Cargando modelo preentrenado BETO...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

...

BLEU calculado sobre 200 muestras aleatorias
Época 40/40 | Tiempo: 0m 28s
Pérdida de entrenamiento: 1.575 | Pérdida de validación: 3.638
Precisión entrenamiento: 0.961 | Precisión validación: 0.677
Perplejidad entrenamiento: 4.830 | Perplejidad validación: 38.004
BLEU validación: 0.2308 | Ratio de sobreajuste: 0.43
Alerta: Sobreajuste severo detectado (ratio entrenamiento/validación < 0.5)

Ejemplos de traducción:
Fuente: El niño tiene la panza resaltada
Destino: Terel xchʼujt alal
Predicción: Busul xchʼuht te alale

Fuente: Está claro lo que dice
Destino: Jahtalal chikan te binti ya kyale
Predicción: Jahtalal chikan te binti ya k yale

Fuente: Ya tengo veinte años
Destino: Ayonix ta jtab jabil
Predicción: Ayonix ta ton te s k


Evaluando modelo con mejor pérdida:


Evaluando:   0%|          | 0/16 [00:00<?, ?it/s]

BLEU calculado sobre 200 muestras aleatorias

Evaluando modelo con mejor BLEU:


Evaluando:   0%|          | 0/16 [00:00<?, ?it/s]

BLEU calculado sobre 200 muestras aleatorias

Resultados en conjunto de prueba:
  Modelo con mejor loss: Loss=3.408, BLEU=0.2500
  Modelo con mejor BLEU: Loss=3.408, BLEU=0.2233

Modelo final: modelo con mejor loss

Mejoras con técnicas anti-sobreajuste y transfer learning:
  BLEU final: 0.2500
  Accuracy final: 0.707
  Perplejidad final: 30.193

Ejemplos de traducción del modelo final:
Fuente: El niño tiene la panza resaltada
Destino: Terel xchʼujt alal
Predicción: Busul xchʼuht te alale

Fuente: Está claro lo que dice
Destino: Jahtalal chikan te binti ya kyale
Predicción: Jahtalal chikan te binti ya k yale

Fuente: Ya tengo veinte años
Destino: Ayonix ta jtab jabil
Predicción: Ayonix ta jtab jabil

Fuente: Viene de vez en cuando
Destino: Namik namik to ya xtal
Predicción: Ba kʼintik ya xtal

Fuente: La mujer tiene la nariz retorcida por enojo
Destino: Buchul sniʼ ta ilimba te antse
Predicción: Buchul sniʼ yuʼun ilimba te antse

Entrenamiento completado!


# Testing

In [18]:
import os
import torch
import re
import json
from tokenizers import Tokenizer
from tqdm import tqdm
import pandas as pd

# CONFIGURACIÓN
BASE_PATH = '/content/drive/MyDrive/MayanV'
MODEL_PATH = 'model_tzh_best_bleu.pt'  # El modelo entrenado en tzh
TOKENIZER_SRC_PATH = 'tokenizers_tzh/spanish_bpe.json'
TOKENIZER_TGT_PATH = 'tokenizers_tzh/tzh_bpe.json'
SAVE_RESULTS = True  # Guardar resultados en CSV

# Funciones y clases desde el código existente
def normalize_string(s):
    s = re.sub(r'#\w+#\s*', '', s)
    return s.strip()

class Config:
    EMBEDDING_DIM = 256
    HIDDEN_DIM = 256
    N_LAYERS = 2
    DROPOUT = 0.45
    MAX_LENGTH = 50
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class BPEVocabulary:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.index2word = {}

        vocab = json.loads(tokenizer.to_str())["model"]["vocab"]
        for token, id in vocab.items():
            self.index2word[id] = token

        self.pad_id = tokenizer.token_to_id("<PAD>")
        self.sos_id = tokenizer.token_to_id("[SOS]")
        self.eos_id = tokenizer.token_to_id("[EOS]")
        self.unk_id = tokenizer.token_to_id("<UNK>")

        self.n_words = len(vocab)

    def encode(self, text):
        encoding = self.tokenizer.encode(text)
        return encoding.ids

    def decode(self, ids):
        return self.tokenizer.decode(ids)

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim, hidden_dim, n_layers,
                         dropout=dropout, batch_first=True, bidirectional=True)
        self.fc = torch.nn.Linear(hidden_dim * 2, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        self.n_layers = n_layers

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)

        # Reorganizar el estado oculto
        hidden_forward = hidden[0:hidden.size(0):2]
        hidden_backward = hidden[1:hidden.size(0):2]

        hidden_transformed = []
        for i in range(self.n_layers):
            combined = torch.cat((hidden_forward[i], hidden_backward[i]), dim=1)
            transformed = torch.tanh(self.fc(combined))
            hidden_transformed.append(transformed.unsqueeze(0))

        final_hidden = torch.cat(hidden_transformed, dim=0)

        return outputs, final_hidden

class Attention(torch.nn.Module):
    def __init__(self, hidden_dim, method='general'):
        super().__init__()
        self.method = method
        self.hidden_dim = hidden_dim

        if self.method == 'general':
            self.attn = torch.nn.Linear(hidden_dim*2, hidden_dim)
            self.v = torch.nn.Linear(hidden_dim, 1, bias=False)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(hidden_dim*3, hidden_dim)
            self.v = torch.nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        hidden = hidden.transpose(0, 1)
        hidden = hidden.repeat(1, src_len, 1)

        if self.method == 'general':
            energy = torch.tanh(self.attn(encoder_outputs))
            energy = self.v(energy).squeeze(2)

        elif self.method == 'concat':
            energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
            energy = self.v(energy).squeeze(2)

        return torch.nn.functional.softmax(energy, dim=1)

class Decoder(torch.nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = torch.nn.Embedding(output_dim, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim + hidden_dim*2, hidden_dim, n_layers,
                         dropout=dropout, batch_first=True)
        self.fc_out = torch.nn.Linear(hidden_dim*3 + embedding_dim, output_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))

        attn_weights = self.attention(hidden, encoder_outputs)
        attn_weights = attn_weights.unsqueeze(1)

        context = torch.bmm(attn_weights, encoder_outputs)

        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)

        embedded = embedded.squeeze(1)
        output = output.squeeze(1)
        context = context.squeeze(1)

        prediction = self.fc_out(torch.cat((output, context, embedded), dim=1))

        return prediction, hidden

class Seq2Seq(torch.nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.7):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        input = tgt[:, 0]

        for t in range(1, tgt_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)

            outputs[:, t, :] = output

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = tgt[:, t] if teacher_force else top1

        return outputs

def translate_sentence(sentence, src_vocab, tgt_vocab, model, device, max_length=Config.MAX_LENGTH):
    model.eval()

    sentence = normalize_string(sentence)
    src_ids = src_vocab.encode(sentence)

    if len(src_ids) > max_length:
        src_ids = src_ids[:max_length]

    src_tensor = torch.LongTensor(src_ids).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

    input = torch.tensor([tgt_vocab.sos_id]).to(device)

    generated_ids = []

    for _ in range(max_length):
        with torch.no_grad():
            output, hidden = model.decoder(input, hidden, encoder_outputs)

        pred_token = output.argmax(1).item()

        if pred_token == tgt_vocab.eos_id:
            break

        generated_ids.append(pred_token)

        input = torch.tensor([pred_token]).to(device)

    return tgt_vocab.decode(generated_ids)

def simple_word_match(reference, hypothesis):
    """Un cálculo simple de coincidencia de palabras en vez de BLEU"""
    ref_words = set(reference.lower().split())
    hyp_words = set(hypothesis.lower().split())
    common_words = ref_words.intersection(hyp_words)
    if len(ref_words) == 0 or len(hyp_words) == 0:
        return 0
    precision = len(common_words) / len(hyp_words) if len(hyp_words) > 0 else 0
    recall = len(common_words) / len(ref_words) if len(ref_words) > 0 else 0
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)  # F1 score

def main():
    print(f"Usando dispositivo: {Config.DEVICE}")

    # Cargar tokenizadores
    print("Cargando tokenizadores...")
    tokenizer_src = Tokenizer.from_file(TOKENIZER_SRC_PATH)
    tokenizer_tgt = Tokenizer.from_file(TOKENIZER_TGT_PATH)

    # Crear vocabularios
    es_vocab = BPEVocabulary(tokenizer_src)
    tzh_vocab = BPEVocabulary(tokenizer_tgt)

    # Cargar modelo entrenado
    print("Cargando modelo TZH...")

    # Crear componentes del modelo
    encoder = Encoder(
        input_dim=es_vocab.n_words,
        embedding_dim=Config.EMBEDDING_DIM,
        hidden_dim=Config.HIDDEN_DIM,
        n_layers=Config.N_LAYERS,
        dropout=Config.DROPOUT
    )

    attention = Attention(
        hidden_dim=Config.HIDDEN_DIM
    )

    decoder = Decoder(
        output_dim=tzh_vocab.n_words,
        embedding_dim=Config.EMBEDDING_DIM,
        hidden_dim=Config.HIDDEN_DIM,
        n_layers=Config.N_LAYERS,
        dropout=Config.DROPOUT,
        attention=attention
    )

    # Crear modelo Seq2Seq
    model = Seq2Seq(encoder, decoder, Config.DEVICE).to(Config.DEVICE)

    # Cargar los pesos entrenados
    model.load_state_dict(torch.load(MODEL_PATH, map_location=Config.DEVICE))
    model.eval()

    # Cargar ejemplos del conjunto de prueba TZH
    print("Cargando datos de prueba para TZH...")
    test_path = os.path.join(BASE_PATH, 'tzh', 'test')
    es_file = os.path.join(test_path, 'data.es')
    tzh_file = os.path.join(test_path, 'data.tzh')

    # Leer archivos
    with open(es_file, 'r', encoding='utf-8') as f:
        es_lines = [normalize_string(line.strip()) for line in f.readlines()]

    with open(tzh_file, 'r', encoding='utf-8') as f:
        tzh_lines = [line.strip() for line in f.readlines()]

    # Asegurar que tenemos el mismo número de líneas
    if len(es_lines) != len(tzh_lines):
        min_lines = min(len(es_lines), len(tzh_lines))
        es_lines = es_lines[:min_lines]
        tzh_lines = tzh_lines[:min_lines]

    # Traducir y evaluar
    results = []

    print(f"Traduciendo {len(es_lines)} ejemplos de prueba...")
    for i, (src, real_tzh) in enumerate(tqdm(zip(es_lines, tzh_lines))):
        # Traducir usando el modelo TZH
        pred_tzh = translate_sentence(src, es_vocab, tzh_vocab, model, Config.DEVICE)

        # Calcular coincidencia exacta
        exact_match = 1 if pred_tzh.strip() == real_tzh.strip() else 0

        # Usar coincidencia de palabras simple en lugar de BLEU
        score = simple_word_match(real_tzh, pred_tzh)

        results.append({
            'ID': i + 1,
            'Español': src,
            'TZH Real': real_tzh,
            'TZH Predicción': pred_tzh,
            'Coincidencia': score,
            'Exacta': exact_match
        })

    # Mostrar todas las traducciones
    df = pd.DataFrame(results)

    # Guardar resultados a CSV
    if SAVE_RESULTS:
        output_file = 'todas_las_traducciones_tzh.csv'
        df.to_csv(output_file, index=False)
        print(f"Resultados guardados en {output_file}")

    # Mostrar todas las traducciones sin limite
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    print("\nTodas las traducciones:")
    display(df[['ID', 'Español', 'TZH Real', 'TZH Predicción', 'Coincidencia', 'Exacta']])

    # Calcular estadísticas globales
    exact_match_rate = df['Exacta'].mean()
    avg_score = df['Coincidencia'].mean()

    print(f"\nTasa de coincidencia exacta: {exact_match_rate:.4f} ({df['Exacta'].sum()} de {len(df)})")
    print(f"Puntuación promedio de coincidencia: {avg_score:.4f}")

    return df

if __name__ == "__main__":
    df = main()

Usando dispositivo: cuda
Cargando tokenizadores...
Cargando modelo TZH...
Cargando datos de prueba para TZH...
Traduciendo 1000 ejemplos de prueba...


1000it [00:08, 114.49it/s]


Resultados guardados en todas_las_traducciones_tzh.csv

Todas las traducciones:


,ID,Español,TZH Real,TZH Predicción,Coincidencia,Exacta
0,1,El niño tiene la panza resaltada,Terel xchʼujt alal,Busul xchʼuht te alale,0.000000,0
1,2,Está claro lo que dice,Jahtalal chikan te binti ya kyale,Jahtalal chikan te binti ya k yale,0.769231,0
2,3,Ya tengo veinte años,Ayonix ta jtab jabil,Ayonix ta jtab jabil,1.000000,1
3,4,Viene de vez en cuando,Namik namik to ya xtal,Ba kʼintik ya xtal,0.500000,0
4,5,La mujer tiene la nariz retorcida por enojo,Buchul sniʼ ta ilimba te antse,Buchul sniʼ yuʼun ilimba te antse,0.833333,0
5,6,El niño camina cojo,Kutsʼkon ta bel jaʼ i tut keremi,Ya yichʼ ch yuchʼ te alale,0.000000,0
6,7,"Se va a hacinar la leña, para que se seque","Ya xlahtsaj te siʼe, yuʼun ya xtakij a","Ya xlah tsaj te siʼe , yuʼun ya xtakij a",0.625000,0
7,8,En el zacatal hay muchas serpientes,Bayal chan ta akiltik,Bayal chan ta akiltik,1.000000,1
8,9,No me jales el cabello,Maʼ abutsbon jol,Ma xa sel ʼakʼ tayon,0.000000,0
9,10,La mujer se fue a comprar en la calle,Baht ta mambajel ta kaya me antse,Baht ta mambajel ta kaya me antse,1.000000,1



Tasa de coincidencia exacta: 0.1620 (162 de 1000)
Puntuación promedio de coincidencia: 0.5075


In [23]:
import torch
import numpy as np
from scipy.stats import entropy
from sklearn.metrics.pairwise import cosine_similarity
from tokenizers import Tokenizer
import json
import os

# Constantes de configuración
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 50

# ================ UTILIDADES BÁSICAS ================

def normalize_string(s):
    """Normaliza texto eliminando etiquetas especiales"""
    import re
    s = re.sub(r'#\w+#\s*', '', s)
    return s.strip()

class BPEVocabulary:
    """Clase para manejar vocabularios BPE"""
    def __init__(self, tokenizer_path):
        self.tokenizer = Tokenizer.from_file(tokenizer_path)
        self.index2word = {}

        # Crear mapeo inverso
        vocab = json.loads(self.tokenizer.to_str())["model"]["vocab"]
        for token, id in vocab.items():
            self.index2word[id] = token

        # IDs de tokens especiales
        self.pad_id = self.tokenizer.token_to_id("<PAD>")
        self.sos_id = self.tokenizer.token_to_id("[SOS]")
        self.eos_id = self.tokenizer.token_to_id("[EOS]")
        self.unk_id = self.tokenizer.token_to_id("<UNK>")

        # Tamaño del vocabulario
        self.n_words = len(vocab)

    def encode(self, text):
        """Codifica texto a lista de IDs"""
        encoding = self.tokenizer.encode(text)
        return encoding.ids

    def decode(self, ids):
        """Decodifica lista de IDs a texto"""
        return self.tokenizer.decode(ids)

# ================ CARGA DE MODELO ================

def load_model(model_path, es_vocab_path, tzh_vocab_path):
    """Carga el modelo entrenado y los vocabularios"""
    # Cargar vocabularios
    es_vocab = BPEVocabulary(es_vocab_path)
    tzh_vocab = BPEVocabulary(tzh_vocab_path)

    # Definir arquitectura (necesario para cargar los pesos)
    encoder = Encoder(
        input_dim=es_vocab.n_words,
        embedding_dim=256,
        hidden_dim=256,
        n_layers=2,
        dropout=0  # Dropout 0 para evaluación
    )

    attention = Attention(hidden_dim=256)

    decoder = Decoder(
        output_dim=tzh_vocab.n_words,
        embedding_dim=256,
        hidden_dim=256,
        n_layers=2,
        dropout=0,  # Dropout 0 para evaluación
        attention=attention
    )

    # Crear modelo y cargar pesos
    model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.eval()  # Modo evaluación

    return model, es_vocab, tzh_vocab

# ================ CLASES DEL MODELO ================

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim, hidden_dim, n_layers,
                         batch_first=True, bidirectional=True)
        self.fc = torch.nn.Linear(hidden_dim * 2, hidden_dim)
        self.dropout = torch.nn.Dropout(dropout)
        self.n_layers = n_layers

    def forward(self, src):
        # Los dropouts no tienen efecto en modo eval
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)

        # Reorganizar el estado oculto
        hidden_forward = hidden[0:hidden.size(0):2]
        hidden_backward = hidden[1:hidden.size(0):2]

        hidden_transformed = []
        for i in range(self.n_layers):
            combined = torch.cat((hidden_forward[i], hidden_backward[i]), dim=1)
            transformed = torch.tanh(self.fc(combined))
            hidden_transformed.append(transformed.unsqueeze(0))

        final_hidden = torch.cat(hidden_transformed, dim=0)
        return outputs, final_hidden

class Attention(torch.nn.Module):
    def __init__(self, hidden_dim, method='general'):
        super().__init__()
        self.method = method
        self.hidden_dim = hidden_dim

        if self.method == 'general':
            self.attn = torch.nn.Linear(hidden_dim*2, hidden_dim)
            self.v = torch.nn.Linear(hidden_dim, 1, bias=False)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(hidden_dim*3, hidden_dim)
            self.v = torch.nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        hidden = hidden.transpose(0, 1)
        hidden = hidden.repeat(1, src_len, 1)

        if self.method == 'general':
            energy = torch.tanh(self.attn(encoder_outputs))
            energy = self.v(energy).squeeze(2)
        elif self.method == 'concat':
            energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
            energy = self.v(energy).squeeze(2)

        return torch.nn.functional.softmax(energy, dim=1)

class Decoder(torch.nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = torch.nn.Embedding(output_dim, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim + hidden_dim*2, hidden_dim, n_layers,
                         batch_first=True)
        self.fc_out = torch.nn.Linear(hidden_dim*3 + embedding_dim, output_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))

        attn_weights = self.attention(hidden, encoder_outputs)
        attn_weights = attn_weights.unsqueeze(1)

        context = torch.bmm(attn_weights, encoder_outputs)

        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)

        embedded = embedded.squeeze(1)
        output = output.squeeze(1)
        context = context.squeeze(1)

        prediction = self.fc_out(torch.cat((output, context, embedded), dim=1))

        return prediction, hidden, attn_weights.squeeze(1)  # También devolvemos los pesos de atención

class Seq2Seq(torch.nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0):
        # No usamos el forward para interpretabilidad
        pass

    def translate(self, src_tensor, src_vocab, tgt_vocab, max_length=MAX_LENGTH):
        """Método para traducir una entrada y obtener todos los datos intermedios"""
        self.eval()

        # Datos a recopilar
        attention_weights = []
        hidden_states = []
        predictions = []
        token_probs = []

        # Codificar entrada
        with torch.no_grad():
            encoder_outputs, hidden = self.encoder(src_tensor)
            hidden_states.append(hidden.detach().cpu().numpy())

            # Decodificar paso a paso
            input_token = torch.tensor([tgt_vocab.sos_id]).to(self.device)

            # Guardar tokens generados
            generated_ids = []

            for t in range(max_length):
                # Decodificar un token
                output, hidden, attn_weight = self.decoder(input_token, hidden, encoder_outputs)

                # Guardar datos de este paso
                attention_weights.append(attn_weight.detach().cpu().numpy())
                hidden_states.append(hidden.detach().cpu().numpy())

                # Calcular probabilidades de token
                token_prob = torch.nn.functional.softmax(output, dim=1)
                token_probs.append(token_prob.detach().cpu().numpy())

                # Obtener predicción
                pred_token = output.argmax(1).item()
                predictions.append(pred_token)

                # Si es EOS, terminar
                if pred_token == tgt_vocab.eos_id:
                    break

                # Añadir a lista de IDs generados
                generated_ids.append(pred_token)

                # Actualizar entrada para siguiente paso
                input_token = torch.tensor([pred_token]).to(self.device)

        # Decodificar tokens a texto
        translation = tgt_vocab.decode(generated_ids)

        # Obtener tokens fuente
        src_ids = src_tensor.squeeze(0).detach().cpu().numpy()
        src_tokens = [src_vocab.index2word.get(id, "<UNK>") for id in src_ids]

        # Obtener tokens destino
        tgt_tokens = ["[SOS]"] + [tgt_vocab.index2word.get(id, "<UNK>") for id in generated_ids]

        # Recopilar todos los datos de interpretabilidad
        interpretation_data = {
            "translation": translation,
            "attention_weights": np.vstack(attention_weights),
            "hidden_states": hidden_states,
            "token_predictions": predictions,
            "token_probabilities": token_probs,
            "source_tokens": src_tokens,
            "target_tokens": tgt_tokens,
            "encoder_states": encoder_outputs.detach().cpu().numpy()
        }

        return interpretation_data

# ================ EXTRACTOR DE CARACTERÍSTICAS DE INTERPRETABILIDAD ================

class InterpretabilityExtractor:
    """Clase para extraer datos de interpretabilidad del modelo"""

    def __init__(self, model, es_vocab, tzh_vocab, device=DEVICE):
        self.model = model
        self.src_vocab = es_vocab
        self.tgt_vocab = tzh_vocab
        self.device = device
        self.model.eval()  # Asegurar modo evaluación

    def preprocess_sentence(self, sentence):
        """Preprocesa y tokeniza una oración para el modelo"""
        sentence = normalize_string(sentence)
        src_ids = self.src_vocab.encode(sentence)
        src_tensor = torch.LongTensor(src_ids).unsqueeze(0).to(self.device)
        return src_tensor

    def analyze_sentence(self, sentence):
        """Analiza una oración y extrae todos los datos de interpretabilidad"""
        src_tensor = self.preprocess_sentence(sentence)
        return self.model.translate(src_tensor, self.src_vocab, self.tgt_vocab)

    def analyze_batch(self, sentences):
        """Analiza un lote de oraciones y extrae características de interpretabilidad"""
        results = []
        for sentence in sentences:
            results.append(self.analyze_sentence(sentence))
        return results

    def extract_neuron_activations(self, sentence, layer=0):
        """Extrae activaciones de neuronas específicas para una oración"""
        src_tensor = self.preprocess_sentence(sentence)

        with torch.no_grad():
            encoder_outputs, hidden = self.model.encoder(src_tensor)

            # Activaciones de neuronas - capa específica
            layer_activations = hidden[layer, 0].cpu().numpy()

            # Activaciones medias por token
            token_activations = encoder_outputs.mean(dim=1).cpu().numpy()[0]

            # Separar direcciones forward y backward
            forward_activations = token_activations[:len(token_activations)//2]
            backward_activations = token_activations[len(token_activations)//2:]

        return {
            "layer_activations": layer_activations,
            "forward_activations": forward_activations,
            "backward_activations": backward_activations,
            "all_activations": token_activations
        }

    def extract_embedding_similarities(self, words):
        """Calcula similitud entre embeddings de palabras dadas"""
        # Obtener índices de palabras
        word_ids = []
        valid_words = []

        for word in words:
            encoded = self.src_vocab.encode(word)
            if len(encoded) == 1:  # Solo palabras que son un único token
                word_ids.append(encoded[0])
                valid_words.append(word)

        if not word_ids:
            return None, []

        # Obtener embeddings
        with torch.no_grad():
            word_tensor = torch.LongTensor(word_ids).to(self.device)
            embeddings = self.model.encoder.embedding(word_tensor).cpu().numpy()

        # Calcular similitud de coseno
        similarity_matrix = cosine_similarity(embeddings)

        return similarity_matrix, valid_words

    def analyze_attention_patterns(self, sentences):
        """Analiza patrones estadísticos de atención en un conjunto de oraciones"""
        attention_stats = []

        for sentence in sentences:
            data = self.analyze_sentence(sentence)
            attention_matrix = data["attention_weights"]

            # Estadísticas por oración
            stats = {
                "sentence": sentence,
                "translation": data["translation"],
                # 1. Focalización (concentración) - mayor valor de atención promedio
                "attention_max": np.max(attention_matrix),
                # 2. Entropía (menor = más concentrada)
                "attention_entropy": np.mean([entropy(row) for row in attention_matrix]),
                # 3. Distribución de atención
                "attention_mean": np.mean(attention_matrix),
                "attention_std": np.std(attention_matrix),
                # 4. Fuerza diagonal (alineamiento monotónico)
                "diagonal_strength": self._calc_diagonal_strength(attention_matrix),
                # 5. Posiciones de máxima atención
                "max_positions": np.argmax(attention_matrix, axis=1).tolist()
            }

            attention_stats.append(stats)

        return attention_stats

    def _calc_diagonal_strength(self, attention_matrix):
        """Calcula la fuerza de la diagonal en la matriz de atención"""
        rows, cols = min(attention_matrix.shape[0], attention_matrix.shape[1]), min(attention_matrix.shape[0], attention_matrix.shape[1])
        if rows == 0 or cols == 0:
            return 0

        diagonal_mask = np.eye(rows, cols, dtype=bool)
        return np.mean(attention_matrix[:rows, :cols][diagonal_mask])

    def calculate_contextual_word_similarity(self, word, context_sentences):
        """Analiza similitud contextual de una palabra en diferentes oraciones"""
        word_representations = []
        valid_sentences = []

        for sentence in context_sentences:
            # Verificar que la palabra está en la oración
            if word not in sentence.split():
                continue

            src_tensor = self.preprocess_sentence(sentence)
            src_ids = src_tensor.squeeze(0).cpu().numpy().tolist()

            # Localizar la palabra en los tokens
            src_tokens = [self.src_vocab.index2word.get(id, "<UNK>") for id in src_ids]
            word_positions = [j for j, token in enumerate(src_tokens) if token == word]

            if not word_positions:
                continue

            valid_sentences.append(sentence)

            # Obtener representaciones contextuales
            with torch.no_grad():
                encoder_outputs, _ = self.model.encoder(src_tensor)

                for pos in word_positions:
                    if pos < encoder_outputs.size(1):
                        representation = encoder_outputs[0, pos, :].cpu().numpy()
                        word_representations.append(representation)

        if len(word_representations) < 2:
            return None, []

        # Calcular similitud entre representaciones
        similarity_matrix = cosine_similarity(word_representations)

        return similarity_matrix, valid_sentences

# ================ EXTRACCIÓN DE DATOS ================

def extract_interpretability_data(model_path, es_vocab_path, tzh_vocab_path, sentences, word_list=None, context_words=None):
    """Función principal para extraer datos de interpretabilidad"""

    # Cargar modelo y vocabularios
    model, es_vocab, tzh_vocab = load_model(model_path, es_vocab_path, tzh_vocab_path)

    # Crear extractor
    extractor = InterpretabilityExtractor(model, es_vocab, tzh_vocab)

    # Recopilar datos de interpretabilidad
    interpretability_data = {
        "sentence_translations": [],
        "attention_patterns": [],
        "neuron_activations": [],
        "word_embeddings": None,
        "contextual_similarities": {}
    }

    # 1. Analizar atención y traducción
    print("Analizando patrones de atención y traducción...")
    interpretability_data["attention_patterns"] = extractor.analyze_attention_patterns(sentences)

    # 2. Extraer traducciones y datos detallados para cada oración
    print("Extrayendo datos detallados por oración...")
    for sentence in sentences:
        data = extractor.analyze_sentence(sentence)
        interpretability_data["sentence_translations"].append({
            "sentence": sentence,
            "translation": data["translation"],
            "attention_matrix": data["attention_weights"],
            "source_tokens": data["source_tokens"],
            "target_tokens": data["target_tokens"]
        })

    # 3. Extraer activaciones de neuronas
    print("Extrayendo activaciones de neuronas...")
    for sentence in sentences:
        activations = extractor.extract_neuron_activations(sentence)
        interpretability_data["neuron_activations"].append({
            "sentence": sentence,
            "activations": activations
        })

    # 4. Calcular similitud de embeddings (opcional)
    if word_list:
        print("Calculando similitud entre embeddings de palabras...")
        similarity_matrix, valid_words = extractor.extract_embedding_similarities(word_list)
        if similarity_matrix is not None:
            interpretability_data["word_embeddings"] = {
                "words": valid_words,
                "similarity_matrix": similarity_matrix
            }

    # 5. Calcular similitud contextual (opcional)
    if context_words:
        print("Calculando similitud contextual de palabras...")
        for word, contexts in context_words.items():
            similarity_matrix, valid_sentences = extractor.calculate_contextual_word_similarity(word, contexts)
            if similarity_matrix is not None:
                interpretability_data["contextual_similarities"][word] = {
                    "sentences": valid_sentences,
                    "similarity_matrix": similarity_matrix
                }

    print("Extracción de datos completada.")
    return interpretability_data

# ================ EJEMPLO DE USO ================

def main():
    # Rutas de archivos
    model_path = 'model_tzh_best_bleu.pt'
    es_vocab_path = 'tokenizers_tzh/spanish_bpe.json'
    tzh_vocab_path = 'tokenizers_tzh/tzh_bpe.json'

    # Oraciones de ejemplo para análisis
    sentences = [
        "Hola, ¿cómo estás?",
        "El perro corre en el parque",
        "Me gusta la comida tradicional",
        "Los niños están jugando afuera",
        "Necesito ir al mercado a comprar frutas"
    ]

    # Palabras para análisis de embeddings
    word_list = ['perro', 'gato', 'casa', 'comida', 'niño', 'familia']

    # Palabras para análisis contextual
    context_words = {
        'casa': [
            "La casa es grande y bonita",
            "Vamos a casa ahora",
            "Mi casa está cerca del mercado",
            "La casa de mi abuela es antigua"
        ]
    }

    # Extraer datos de interpretabilidad
    data = extract_interpretability_data(
        model_path,
        es_vocab_path,
        tzh_vocab_path,
        sentences,
        word_list,
        context_words
    )

    # Guardar datos para posterior visualización
    import pickle
    os.makedirs('interpretability_data', exist_ok=True)
    with open('interpretability_data/extracted_data.pkl', 'wb') as f:
        pickle.dump(data, f)

    print("Datos guardados en 'interpretability_data/extracted_data.pkl'")
    return data

if __name__ == "__main__":
    main()

Analizando patrones de atención y traducción...
Extrayendo datos detallados por oración...
Extrayendo activaciones de neuronas...
Calculando similitud entre embeddings de palabras...
Calculando similitud contextual de palabras...
Extracción de datos completada.
Datos guardados en 'interpretability_data/extracted_data.pkl'


In [25]:
import torch
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import pickle
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from IPython.display import display, HTML

# Configuración de estilo para las visualizaciones
PLOTLY_THEME = "plotly_white"  # Tema limpio y elegante
COLOR_SCALE = "RdBu_r"  # Escala rojo-azul invertida
ACCENT_COLOR = "#1f77b4"  # Azul principal
SECONDARY_COLOR = "#ff7f0e"  # Naranja
FONT_FAMILY = "Arial, sans-serif"

# Función para mostrar la visualización directamente en Colab
def show_visualization(fig, title=None):
    """Muestra una figura de plotly en el notebook con estilo mejorado"""
    # Aplicar tema y estilo base
    fig.update_layout(
        template=PLOTLY_THEME,
        font=dict(family=FONT_FAMILY),
        margin=dict(l=40, r=40, t=80, b=40),
        legend=dict(
            bordercolor="lightgray",
            borderwidth=1,
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Añadir título personalizado si se proporciona
    if title:
        fig.update_layout(
            title=dict(
                text=title,
                font=dict(size=22, color="#333"),
                x=0.5,
                xanchor="center"
            )
        )

    # Mostrar la figura
    display(HTML(f"<div style='border-left: 5px solid {ACCENT_COLOR}; padding-left: 15px;'>"))
    fig.show()
    display(HTML("</div>"))

    return fig

# Visualización 1: Mapa de atención mejorado
def create_attention_heatmap(attention_matrix, src_tokens, tgt_tokens, sentence, translation):
    """Crea un mapa de calor de atención mejorado"""

    # Crear figura
    fig = go.Figure()

    # Añadir heatmap con mejor diseño
    fig.add_trace(go.Heatmap(
        z=attention_matrix,
        x=src_tokens,
        y=tgt_tokens,
        colorscale="Blues",
        hoverongaps=False,
        colorbar=dict(
            title="Peso de atención",
            titleside="right",
            thickness=15,
            len=0.7,
            outlinewidth=1,
            outlinecolor="lightgray"
        ),
        hovertemplate="<b>Fuente:</b> %{x}<br><b>Destino:</b> %{y}<br><b>Atención:</b> %{z:.4f}<extra></extra>"
    ))

    # Mejorar el diseño
    fig.update_layout(
        title=f"<b>Mapa de atención</b><br><span style='font-size:16px'>'{sentence}'</span><br><span style='font-size:14px; color:gray'>Traducción: '{translation}'</span>",
        xaxis=dict(
            title="Tokens de entrada (español)",
            tickangle=45,
            tickfont=dict(size=12),
            showgrid=False,
            showline=True,
            linecolor="lightgray"
        ),
        yaxis=dict(
            title="Tokens generados (tzh)",
            autorange="reversed",
            tickfont=dict(size=12),
            showgrid=False,
            showline=True,
            linecolor="lightgray"
        ),
        height=500,
        width=700
    )

    return fig

# Visualización 2: Comparación interactiva de atención para múltiples oraciones
def create_multi_sentence_comparison(data):
    """Crea un panel interactivo para comparar atención en múltiples oraciones"""

    # Extraer datos
    sentences = []
    translations = []
    attention_matrices = []
    src_tokens_list = []
    tgt_tokens_list = []

    for item in data["sentence_translations"]:
        sentences.append(item["sentence"])
        translations.append(item["translation"])
        attention_matrices.append(item["attention_matrix"])
        src_tokens_list.append(item["source_tokens"])
        tgt_tokens_list.append(item["target_tokens"])

    # Crear figura base con el primer mapa de atención
    fig = go.Figure()

    # Añadir el primer heatmap
    fig.add_trace(go.Heatmap(
        z=attention_matrices[0],
        x=src_tokens_list[0],
        y=tgt_tokens_list[0],
        colorscale="Blues",
        hoverongaps=False,
        colorbar=dict(
            title="Peso de atención",
            thickness=15,
            len=0.7
        )
    ))

    # Añadir selector mediante dropdown
    dropdown_buttons = []
    for i, (sentence, translation) in enumerate(zip(sentences, translations)):
        dropdown_buttons.append(
            dict(
                method="update",
                label=f"<b>Oración {i+1}</b>: {sentence[:30]}...",
                args=[
                    {
                        "z": [attention_matrices[i]],
                        "x": [src_tokens_list[i]],
                        "y": [tgt_tokens_list[i]]
                    },
                    {
                        "title": f"<b>Mapa de atención</b><br><span style='font-size:16px'>'{sentences[i]}'</span><br><span style='font-size:14px; color:gray'>Traducción: '{translations[i]}'</span>"
                    }
                ]
            )
        )

    # Añadir dropdown al layout
    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=dropdown_buttons,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.5,
                xanchor="center",
                y=1.15,
                yanchor="top",
                bgcolor="white",
                font=dict(color="black")
            )
        ]
    )

    # Mejorar el diseño general
    fig.update_layout(
        title=f"<b>Mapa de atención</b><br><span style='font-size:16px'>'{sentences[0]}'</span><br><span style='font-size:14px; color:gray'>Traducción: '{translations[0]}'</span>",
        xaxis=dict(
            title="Tokens de entrada (español)",
            tickangle=45,
            showgrid=False
        ),
        yaxis=dict(
            title="Tokens generados (tzh)",
            autorange="reversed",
            showgrid=False
        ),
        height=600,
        width=800
    )

    return fig

# Visualización 3: Patrón de activación neuronal mejorado
def create_neuron_activation_chart(activations, sentence):
    """Crea una visualización mejorada para activaciones neuronales"""

    # Separar activaciones direccionales (forward y backward)
    half_point = len(activations) // 2
    forward_activations = activations[:half_point]
    backward_activations = activations[half_point:]

    # Crear dataframe para mejor manipulación
    df_forward = pd.DataFrame({
        'Neurona': list(range(len(forward_activations))),
        'Activación': forward_activations,
        'Dirección': ['Forward'] * len(forward_activations)
    })

    df_backward = pd.DataFrame({
        'Neurona': list(range(len(backward_activations))),
        'Activación': backward_activations,
        'Dirección': ['Backward'] * len(backward_activations)
    })

    df = pd.concat([df_forward, df_backward])

    # Crear gráfico con mejor estilo
    fig = px.line(
        df, x='Neurona', y='Activación', color='Dirección',
        color_discrete_map={'Forward': ACCENT_COLOR, 'Backward': SECONDARY_COLOR},
        markers=True, line_shape='spline',
        title=f"<b>Activación neuronal</b><br><span style='font-size:16px'>'{sentence}'</span>"
    )

    # Mejorar diseño
    fig.update_traces(
        marker=dict(size=6),
        line=dict(width=2.5),
        hovertemplate="<b>Neurona:</b> %{x}<br><b>Activación:</b> %{y:.4f}<extra>%{fullData.name}</extra>"
    )

    fig.update_layout(
        xaxis=dict(
            title="Índice de neurona",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)'
        ),
        yaxis=dict(
            title="Nivel de activación",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)',
            zeroline=True,
            zerolinecolor='rgba(0,0,0,0.2)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor='rgba(255,255,255,0.8)'
        ),
        height=400,
        width=700
    )

    return fig

# Visualización 4: Mapa de similitud de embedding con anotaciones
def create_word_similarity_heatmap(similarity_matrix, words):
    """Crea un mapa de calor para visualizar similitud entre embeddings de palabras"""

    # Redondear valores para la visualización
    z_text = np.around(similarity_matrix, decimals=2)

    # Crear heatmap con anotaciones
    fig = go.Figure(data=go.Heatmap(
        z=similarity_matrix,
        x=words,
        y=words,
        colorscale='RdBu_r',
        zmid=0.5,  # Color central en 0.5 (similitud media)
        text=z_text,
        texttemplate="%{text}",
        textfont={"size":10},
        hovertemplate="<b>%{x}</b> y <b>%{y}</b><br>Similitud: %{z:.4f}<extra></extra>"
    ))

    # Mejorar diseño
    fig.update_layout(
        title="<b>Similitud entre embeddings de palabras</b>",
        xaxis=dict(
            title="",
            tickangle=45,
            side="bottom"
        ),
        yaxis=dict(
            title=""
        ),
        height=550,
        width=600
    )

    return fig

# Visualización 5: Proyección 2D estilizada de embeddings
def create_embedding_projection_chart(similarity_matrix, words):
    """Crea una proyección 2D estilizada de embeddings de palabras"""

    # Proyectar embeddings a 2D usando PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(similarity_matrix)

    # Crear dataframe para la visualización
    df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'word': words
    })

    # Crear gráfico de dispersión estilizado
    fig = px.scatter(
        df, x='x', y='y', text='word',
        title="<b>Espacio semántico de palabras</b>",
        opacity=0.8
    )

    # Personalizar puntos y texto
    fig.update_traces(
        marker=dict(
            size=15,
            color=ACCENT_COLOR,
            line=dict(width=1, color='darkslategray')
        ),
        textposition='top center',
        textfont=dict(size=14, color='black')
    )

    # Personalizar ejes
    fig.update_layout(
        xaxis=dict(
            title="Dimensión 1",
            showgrid=True,
            zeroline=True,
            zerolinecolor='rgba(0,0,0,0.2)',
            gridcolor='rgba(0,0,0,0.1)'
        ),
        yaxis=dict(
            title="Dimensión 2",
            showgrid=True,
            zeroline=True,
            zerolinecolor='rgba(0,0,0,0.2)',
            gridcolor='rgba(0,0,0,0.1)'
        ),
        height=500,
        width=700
    )

    return fig

# Visualización 6: Gráfico de radar para estadísticas de atención
def create_attention_stats_radar(attention_stats, sentences):
    """Crea un gráfico de radar para comparar estadísticas de atención"""

    # Extraer valores normalizados para cada métrica
    categories = ['Atención máxima', 'Entropía', 'Atención promedio', 'Fuerza diagonal']

    # Normalizar valores para mejor visualización
    max_values = {
        'attention_max': max([stat['attention_max'] for stat in attention_stats]),
        'attention_entropy': max([stat['attention_entropy'] for stat in attention_stats]),
        'attention_mean': max([stat['attention_mean'] for stat in attention_stats]),
        'diagonal_strength': max([stat['diagonal_strength'] for stat in attention_stats])
    }

    # Crear figura
    fig = go.Figure()

    # Añadir cada oración como un trazo
    for i, stat in enumerate(attention_stats):
        # Extraer valores normalizados
        values = [
            stat['attention_max'] / max_values['attention_max'],
            stat['attention_entropy'] / max_values['attention_entropy'],
            stat['attention_mean'] / max_values['attention_mean'],
            stat['diagonal_strength'] / max_values['diagonal_strength']
        ]

        # Añadir trazo para esta oración
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            name=f"Oración {i+1}",
            line_color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)],
            hoverinfo="text",
            hovertext=[
                f"Atención máxima: {stat['attention_max']:.3f}",
                f"Entropía: {stat['attention_entropy']:.3f}",
                f"Atención promedio: {stat['attention_mean']:.3f}",
                f"Fuerza diagonal: {stat['diagonal_strength']:.3f}"
            ]
        ))

    # Mejorar diseño
    fig.update_layout(
        title="<b>Patrones de atención por oración</b>",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1.2]  # Valores normalizados
            )
        ),
        showlegend=True,
        height=500,
        width=650
    )

    # Añadir anotaciones con frases
    for i, sentence in enumerate(sentences):
        if i < 5:  # Limitar a 5 para evitar sobrecargar
            fig.add_annotation(
                text=f"#{i+1}: '{sentence[:30]}...'",
                x=0,
                y=-0.15 - (i * 0.05),
                xref="paper",
                yref="paper",
                showarrow=False,
                align="left",
                bgcolor=f"rgba{tuple(int(c * 255) for c in px.colors.hex_to_rgb(px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)])) + (0.3,)}",
                bordercolor="lightgray",
                borderwidth=1
            )

    return fig

# Visualización 7: Dashboard completo (atención + activación)
def create_complete_dashboard(data, sentence_index=0):
    """Crea un dashboard completo para una oración específica"""

    # Extraer datos de la oración seleccionada
    sentence_data = data["sentence_translations"][sentence_index]
    activation_data = data["neuron_activations"][sentence_index]["activations"]["all_activations"]

    # Crear subplots
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.6, 0.4],
        vertical_spacing=0.1,
        subplot_titles=(
            "<b>Mapa de atención</b>",
            "<b>Activación neuronal</b>"
        )
    )

    # Agregar mapa de atención
    fig.add_trace(
        go.Heatmap(
            z=sentence_data["attention_matrix"],
            x=sentence_data["source_tokens"],
            y=sentence_data["target_tokens"],
            colorscale="Blues",
            colorbar=dict(
                title="Atención",
                y=0.8,  # Posición vertical
                len=0.5,  # Longitud
                thickness=15  # Grosor
            ),
            hovertemplate="<b>Fuente:</b> %{x}<br><b>Destino:</b> %{y}<br><b>Atención:</b> %{z:.4f}<extra></extra>"
        ),
        row=1, col=1
    )

    # Separar activaciones direccionales
    half_point = len(activation_data) // 2
    forward_activations = activation_data[:half_point]
    backward_activations = activation_data[half_point:]

    # Agregar activaciones forward
    fig.add_trace(
        go.Scatter(
            x=list(range(len(forward_activations))),
            y=forward_activations,
            mode='lines+markers',
            name='Forward',
            line=dict(color=ACCENT_COLOR, width=2),
            marker=dict(size=6)
        ),
        row=2, col=1
    )

    # Agregar activaciones backward
    fig.add_trace(
        go.Scatter(
            x=list(range(len(backward_activations))),
            y=backward_activations,
            mode='lines+markers',
            name='Backward',
            line=dict(color=SECONDARY_COLOR, width=2),
            marker=dict(size=6)
        ),
        row=2, col=1
    )

    # Mejorar diseño
    fig.update_layout(
        title=f"<b>Análisis de traducción</b><br><span style='font-size:16px'>'{sentence_data['sentence']}'</span><br><span style='font-size:14px; color:gray'>Traducción: '{sentence_data['translation']}'</span>",
        height=800,
        width=800
    )

    # Configurar ejes
    fig.update_xaxes(title="Tokens de entrada (español)", tickangle=45, row=1, col=1)
    fig.update_yaxes(title="Tokens generados (tzh)", autorange="reversed", row=1, col=1)
    fig.update_xaxes(title="Índice de neurona", row=2, col=1)
    fig.update_yaxes(title="Nivel de activación", row=2, col=1)

    return fig

# Función principal para mostrar todas las visualizaciones
def display_interpretability_visualizations(data_path='interpretability_data/extracted_data.pkl'):
    """
    Carga los datos de interpretabilidad y muestra visualizaciones directamente en el notebook
    """
    # Cargar datos
    with open(data_path, 'rb') as f:
        data = pickle.load(f)

    # 1. Mostrar mapa de atención para la primera oración
    first_data = data["sentence_translations"][0]
    attention_fig = create_attention_heatmap(
        first_data["attention_matrix"],
        first_data["source_tokens"],
        first_data["target_tokens"],
        first_data["sentence"],
        first_data["translation"]
    )
    show_visualization(attention_fig, "Mapa de atención")

    # 2. Mostrar comparador de múltiples oraciones
    multi_fig = create_multi_sentence_comparison(data)
    show_visualization(multi_fig, "Comparación de atención entre oraciones")

    # 3. Mostrar activación neuronal
    first_activation = data["neuron_activations"][0]["activations"]["all_activations"]
    neuron_fig = create_neuron_activation_chart(first_activation, first_data["sentence"])
    show_visualization(neuron_fig, "Activación neuronal")

    # 4. Mostrar similitud de embeddings
    if data["word_embeddings"] is not None:
        similarity_fig = create_word_similarity_heatmap(
            data["word_embeddings"]["similarity_matrix"],
            data["word_embeddings"]["words"]
        )
        show_visualization(similarity_fig, "Similitud entre embeddings de palabras")

        # 5. Mostrar proyección 2D
        projection_fig = create_embedding_projection_chart(
            data["word_embeddings"]["similarity_matrix"],
            data["word_embeddings"]["words"]
        )
        show_visualization(projection_fig, "Proyección 2D de embeddings")

    # 6. Mostrar estadísticas de atención
    sentences = [item["sentence"] for item in data["sentence_translations"]]
    radar_fig = create_attention_stats_radar(data["attention_patterns"], sentences)
    show_visualization(radar_fig, "Comparación de patrones de atención")

    # 7. Mostrar dashboard completo
    dashboard_fig = create_complete_dashboard(data)
    show_visualization(dashboard_fig, "Dashboard completo de interpretabilidad")

    print(f"\n✅ Se han mostrado todas las visualizaciones interactivas satisfactoriamente.\n")
    return data

# Ejecutar visualizaciones
if __name__ == "__main__":
    display_interpretability_visualizations()


✅ Se han mostrado todas las visualizaciones interactivas satisfactoriamente.

